# 自己位置推定と自律移動

<div class="alert alert-block alert-info">
    <b>この章の目的</b>
    <p>ロボットを指定した場所まで障害物を避けながら自律移動させる方法を学習します</p>
</div> 

# Localization and autonomous movement

<div class="alert alert-block alert-info">
    <b>Objective</b>
    <p>In this notebook, we will learn how to move the robot autonomously to a goal while avoiding other objects.</p>
</div>

# RVizを使った自律移動

# Autonomous movement using RViz

rvizを起動します。下記コマンドを実行して、起動してください。

Execute the following command to start RViz.

In [ ]:
%%script bash --bg
rviz -d data/3_navigation.rviz > /dev/null 2>&1

シミュレータ画面で、rvizが立ち上がったことを確認してください。rvizでは、ロボットに搭載された各種センサ情報を確認できます。

![title](./imgs/3_rviz.png)

In the simulator screen, check that RViz has started. In RViz, you can check various sensor information from the sensors that are mounted on the robot.

![title](./imgs/3_rviz.png)

「Displays」の中のチェックをON, OFFして表示されている情報を確認してみましょう。それぞれの情報の意味は以下の通りです。

- Map: 事前に与えられた地図情報


- global_cost_map: 事前地図を元に生成したコストマップ


- local_cost_map: レーザ情報を元に生成したコストマップ

ここでコストマップとは、通れそうな領域、障害物のある領域を数値で表現した地図のことで、自律移動の際に用いられます。

Let's check the information by turning the checks on and off in the "Displays" panel. The meaning of each item is as follows:

- Map: Predefined map information.

- global_cost_map: Cost map generated based on the predefined map.

- local_cost_map: Cost map generated based on laser scanner information.

Here, the cost map is a map that numerically expresses areas where the robot is likely to pass through or areas with obstacles, and is used for autonomous movement.

rvizから自律移動のゴールを入力します。rvizの上部のツールバーにある「2D Nav Goal」をクリックし、部屋内の障害物のない場所を指定してみましょう。

![title](./imgs/3_2d_nav_goal_button.png)
![title](./imgs/3_2d_nav_goal.png)


HSRが自律移動を開始します。rviz画面や、シミュレータ画面で、HSRが障害物を避けながら指定した位置に移動することを確認しましょう。
- rviz画面
![title](./imgs/3_moving.png)


- gazebo画面
![title](./imgs/3_moving_gazebo.png)

Now, we specify the autonomous movement goal using RViz. Click "2D Nav Goal" in the toolbar at the top of RViz and click an unobstructed location in the room:

![title](./imgs/3_2d_nav_goal_button.png)
![title](./imgs/3_2d_nav_goal.png)

The HSR starts autonomous movement. On the RViz screen or the simulator screen, check that the HSR moves to the specified position while avoiding obstacles.

- RViz screen
![title](./imgs/3_moving.png)

- Gazebo screen
![title](./imgs/3_moving_gazebo.png)

## 障害物回避
次に、人の位置を移動させてみましょう。gazeboの上にある
![title](./imgs/3_move_object_button.png)
このボタンをクリックしてから、人をクリックしてください。
![title](./imgs/3_human_select.png)
青、赤、緑の矢印が表示されました。赤と緑の矢印を動かすと、人をXY平面上にて動かせます。人を動かしている時に、人をロボットにぶつけてまうと、ロボットの挙動がおかしくなるので、ぶつからないように移動させましょう。

終わったら、矢印ボタンを押して、選択状態を終了しましょう。
![title](./imgs/3_back_to_default.png)

## Obstacle avoidance
Next, we will try the obstacle avoidance function.

Click this button at the top of Gazebo:
![title](./imgs/3_move_object_button.png)

then click the person:
![title](./imgs/3_human_select.png)

Blue, red and green arrows are displayed. You can move a person on the XY plane by dragging the red and green arrows. Please do not hit the person against the robot, otherwise the behavior of the robot will get strange.

When you're done, press the arrow button to exit the selection mode:
![title](./imgs/3_back_to_default.png)

rvizの画面に戻って、「2D Nav Goal」を与え、HSRを自律移動させましょう。

![title](./imgs/3_move_avoiding_human.png)

人をよけながら自律移動できたでしょうか？

Return to the RViz screen and use "2D Nav Goal" to move the HSR autonomously.

![title](./imgs/3_move_avoiding_human.png)

You should see the robot move autonomously while avoiding the person.

# Pythonスクリプトからの自律移動

# Autonomous movement using a Python script

前章はrvizから自律移動を実行しましたが、今回はPythonから指定した座標に自律移動する指令を出してみましょう。

In the previous section, autonomous movement was executed from RViz. This time, let's issue a command using a Python script.

## 準備
まず、必要なライブラリをインポートし、初期化します。

## Preparation
First, import and initialize the required libraries:

In [ ]:
import rospy
from utils import *
rospy.init_node('navigation')

## ゴール座標指定
次に、ゴールの座標を決めましょう。rvizの右上の「Publish Point」ボタンをクリックします。
![title](./imgs/3_publish_point_button.png)


次に、地図上で目的地にマウスを持って行くと、rvizの左下の「Select this point」の部分に座標が表示されます。この例では、x=1.29,y=3.13,z=0.00747 が目的地の座標です。zは不要なので、x,yの値をメモしておきましょう。
![title](./imgs/3_point_values.png)

**注意: ゴールを地図から外れた位置に設定すると誤動作するので、地図内の位置にしてください。**

## Specify goal position
Next, let's decide the position of the goal. Click the "Publish Point" button at the top right of RViz:
![title](./imgs/3_publish_point_button.png)

Next, move the mouse to the destination on the map. The coordinates will be displayed in the "Select this point" part at the bottom left of RViz.

In this example, x=1.29, y=3.13, z=0.00747 are the position of the destination. We don't need z, so just note the x and y values:

![title](./imgs/3_point_values.png)

**Note: If you set the goal to a position outside of the map, the robot will malfunction, so please set it to a position within the map.**

## ゴールを送る
ロボットにゴールを送りましょう。下記のコマンドを実行し、シミュレータ上でのロボットの動作を観察してください。

**上級者向け** 台車を自律移動させるには`move_base_goal`という関数を使用します。興味のある人は見てみてください。

```python
# 自律移動のゴールを送信するクライアントを作成
navclient = actionlib.SimpleActionClient('/move_base', MoveBaseAction)

def move_base_goal(x, y, theta):
    u"""台車の自律移動のゴールを指定する関数

    引数：
        x (float): 目標のx値 [m]
        y (float): 目標のy値 [m]
        theta (float): 目標の回転角度 [deg]

    返り値:
        ゴールに到達したらTrue, そうでなければFalse

    """

    goal = MoveBaseGoal()

    # "map"座標を基準座標に指定
    goal.target_pose.header.frame_id = "map"

    # ゴールのx,y座標をセットします
    goal.target_pose.pose.position.x = x
    goal.target_pose.pose.position.y = y

    # 角度はクオータニオンという形式で与えます。そのため、オイラー角からクオータニオンに変換します
    goal.target_pose.pose.orientation = quaternion_from_euler(0, 0, theta)

    # ゴールを送信
    navclient.send_goal(goal)
    navclient.wait_for_result()
    state = navclient.get_state()
    # 成功すると、3が返ってくる
    # http://docs.ros.org/fuerte/api/actionlib_msgs/html/msg/GoalStatus.html
    return True if state == 3 else False
```

以下のコマンドでx=1メートル、y=0.5メートル、theta=90度の位置姿勢に移動できます。

## Send a goal
To send a goal to the robot, execute the following command and observe the movement of the robot in the simulator.

**For advanced users**: To move the move base autonomously, use the function `move_base_goal`. If you are interested, please take a look:

```python
# Create a client to send autonomous move goals
navclient = actionlib.SimpleActionClient ('/move _base', MoveBaseAction)

def move_ base _goal (x, y, theta):
    u"""Function that specifies the goal of autonomous movement of the move base

    argument:
        x(float): Target x value [m]
        y(float): Target y value [m]
        theta(float): Target rotation angle [deg]

    Return value:
        True if you reach the goal, False otherwise
    """

    goal = MoveBaseGoal ()

    # Specify "map" coordinates as reference coordinates
    goal.target_ pose.header.frame _id = "map"

    # Set the x, y coordinates of the goal
    goal.target_pose.pose.position.x = x
    goal.target_pose.pose.position.y = y

    #Angles are given in the form of quaternions. Therefore, convert from Euler angles to quaternions
    goal.target_pose.pose.orientation = quaternion_from_euler(0, 0, theta)

    #Send goal
    navclient.send_goal (goal)
    navclient.wait_for_result ()
    state = navclient.get_state ()
    # If successful, 3 will be returned
    # http://docs.ros.org/fuerte/api/actionlib_msgs/html/msg/GoalStatus.html
    return True if state == 3 else False
`` ```

You can move to the position/orientation of x=1 meter, y=0.5 meter, theta=90 degrees with the following command:

In [ ]:
move_base_goal(1, 0.5, 90)

以下のコマンドで原点に戻ります。

Return to the origin with the following command:

In [ ]:
move_base_goal(0, 0, 0)

<div class="alert alert-block alert-info">
    <b>課題</b>
    <p>ある場所についたら次のゴールに自動的に向かうようにして、部屋を周回するプログラムを書いてみましょう。</p>
</div>

<div class="alert alert-block alert-info">
    <b>Task</b>
    <p>Let's try to write a program that goes around a room, then automatically goes to its next goal after arriving at a certain place.</p>
</div>

In [1]:
# 自分で考えてみましょう。この下に入力できます。 Write your code here:


<div class="alert alert-block alert-info">
    <b>アドバンスド課題</b>
    <p>障害物の上にゴールを置くとどうなるでしょうか？</p>
    <p>自律移動中に人を動かして、ロボットの邪魔をするとどうなるでしょうか？</p>
    <p>地図の外にゴールを置くとどうなるでしょうか</p>
</div>

<div class="alert alert-block alert-info">
    <b>Advanced task</b>
    <p>Check what happens if we set a goal in the obstacle.</p>
    <p>Check what happens if we move a person in front of the robot while the robot is moving.</p>
    <p>Check what happens if we set a goal outside of the map.</p>
</div>

In [2]:
# 自分で考えてみましょう。この下に入力できます。 Write your code here:


<div class="alert alert-block alert-info">
    <b>次の学習</b>
    <p>今までの章では台車の制御を扱ってきました。次の章ではHSRのアームを制御してみましょう。</p>
</div>

<div class="alert alert-block alert-info">
    <b>What's next?</b>
    <p>Now, we can use the mobile base very well. In the next notebook, we will learn how to use the HSR's arm.</p>
</div>